# Introduction to XGBoost Spark with GPU

Taxi is an example of XGBoost regressor. This notebook will show you how to load data, train the XGBoost model and use this model to predict "fare_amount" of your taxi trip.

## Load libraries
First load some common libraries will be used by both GPU version and CPU version XGBoost.

In [1]:
import ml.dmlc.xgboost4j.scala.spark.{XGBoostRegressor, XGBoostRegressionModel}
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.sql.types.{DoubleType, IntegerType, StructField, StructType}

Besides CPU version requires some extra libraries, such as:

```scala
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.FloatType
```

## Set the dataset path

In [7]:
// You need to update them to your real paths! The input data files can be the output of taxi-etl jobs, or you can
// just use the provided sample datasets upder datasets path. 
val dataRoot = sys.env.getOrElse("DATA_ROOT", "/data")
val trainPath = dataRoot + "/taxi/csv/train/"
val evalPath  = dataRoot + "/taxi/csv/test/"
val transPath = dataRoot + "/taxi/csv/test/"

dataRoot = /data
trainPath = /data/taxi/csv/train/
evalPath = /data/taxi/csv/test/
transPath = /data/taxi/csv/test/


/data/taxi/csv/test/

## Build the schema of the dataset
The Taxi data has 16 columns: 15 features and 1 label. "fare_amount" is the label column. The schema will be used to load data in the future. 

The next block also defines some key parameters used in XGBoost training process.

In [8]:
val labelName = "fare_amount"
lazy val schema =
  StructType(Array(
    StructField("vendor_id", DoubleType),
    StructField("passenger_count", DoubleType),
    StructField("trip_distance", DoubleType),
    StructField("pickup_longitude", DoubleType),
    StructField("pickup_latitude", DoubleType),
    StructField("rate_code", DoubleType),
    StructField("store_and_fwd", DoubleType),
    StructField("dropoff_longitude", DoubleType),
    StructField("dropoff_latitude", DoubleType),
    StructField(labelName, DoubleType),
    StructField("hour", DoubleType),
    StructField("year", IntegerType),
    StructField("month", IntegerType),
    StructField("day", DoubleType),
    StructField("day_of_week", DoubleType),
    StructField("is_weekend", DoubleType)
  ))

val featureNames = schema.filter(_.name != labelName).map(_.name).toArray

lazy val paramMap = Map(
  "num_round" -> 100
)

labelName = fare_amount
schema = <lazy>
featureNames = Array(vendor_id, passenger_count, trip_distance, pickup_longitude, pickup_latitude, rate_code, store_and_fwd, dropoff_longitude, dropoff_latitude, hour, year, month, day, day_of_week, is_weekend)
paramMap = <lazy>


<lazy>

## Create a new spark session and load data

A new spark session should be created to continue all the following spark operations.

NOTE: in this notebook, the dependency jars have been loaded when installing toree kernel. Alternatively the jars can be loaded into notebook by [%AddJar magic](https://toree.incubator.apache.org/docs/current/user/faq/). However, there's one restriction for `%AddJar`: the jar uploaded can only be available when `AddJar` is called just after a new spark session is created. Do it as below:

```scala
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().appName("taxi-GPU").getOrCreate
%AddJar file:/data/libs/rapids-4-spark-XXX.jar
%AddJar file:/data/libs/xgboost4j-spark-gpu_2.12-XXX.jar
%AddJar file:/data/libs/xgboost4j-gpu_2.12-XXX.jar
// ...
```

##### Please note the new jar "rapids-4-spark-XXX.jar" is only needed for GPU version, you can not add it to dependence list for CPU version.

In [9]:
// Build the spark session and data reader as usual
val sparkSession = SparkSession.builder().appName("taxi-GPU").getOrCreate
val reader = sparkSession.read.option("header", true).schema(schema)

sparkSession = org.apache.spark.sql.SparkSession@6efbc93b
reader = org.apache.spark.sql.DataFrameReader@64b8d6da


org.apache.spark.sql.DataFrameReader@64b8d6da

In [10]:
// Please make sure to change the api to reader.parquet if you load parquet files.
val trainSet = reader.csv(trainPath)
val evalSet  = reader.csv(evalPath)
val transSet = reader.csv(transPath)

trainSet = [vendor_id: double, passenger_count: double ... 14 more fields]
evalSet = [vendor_id: double, passenger_count: double ... 14 more fields]
transSet = [vendor_id: double, passenger_count: double ... 14 more fields]


[vendor_id: double, passenger_count: double ... 14 more fields]

## Set XGBoost parameters and build a XGBoostRegressor

For CPU version, `num_workers` is recommended being equal to the number of CPU cores, while for GPU version, it should be set to the number of GPUs in Spark cluster.

Besides the `tree_method` for CPU version is also different from that for GPU version. Now only "gpu_hist" is supported for training on GPU.

```scala
// difference in parameters
  "num_workers" -> 12
  "tree_method" -> "hist",

In [11]:
val xgbParamFinal = paramMap ++ Map("tree_method" -> "gpu_hist", "num_workers" -> 1)

xgbParamFinal = Map(num_round -> 100, tree_method -> gpu_hist, num_workers -> 1)


Map(num_round -> 100, tree_method -> gpu_hist, num_workers -> 1)

In [12]:
val xgbRegressor = new XGBoostRegressor(xgbParamFinal)
  .setLabelCol(labelName)
  .setFeaturesCol(featureNames)

xgbRegressor = xgbr_d36c6f5fd67c


xgbr_d36c6f5fd67c

## Benchmark and train
The object `benchmark` is used to compute the elapsed time of some operations.

Training with evaluation dataset is also supported, the same as CPU version's behavior:

* Call API `setEvalDataset` after initializing an XGBoostClassifier

```scala
xgbClassifier.setEvalDataset(evalSet)
```

In [13]:
xgbRegressor.setEvalDataset(evalSet)

xgbr_d36c6f5fd67c

In [14]:
object Benchmark {
  def time[R](phase: String)(block: => R): (R, Float) = {
    val t0 = System.currentTimeMillis
    val result = block // call-by-name
    val t1 = System.currentTimeMillis
    println("Elapsed time [" + phase + "]: " + ((t1 - t0).toFloat / 1000) + "s")
    (result, (t1 - t0).toFloat / 1000)
  }
}

defined object Benchmark


In [15]:
// start training
val (model, _) = Benchmark.time("train") {
  xgbRegressor.fit(trainSet)
}

Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=37275, DMLC_NUM_WORKER=1}


model = xgbr_d36c6f5fd67c


Elapsed time [train]: 7.441s


xgbr_d36c6f5fd67c

## Transformation and evaluation
Here uses `transSet` to evaluate our model and use some key columns to show our predictions. Finally we use `RegressionEvaluator` to calculate an overall `rmse` of our predictions.

In [16]:
// start transform
val (prediction, _) = Benchmark.time("transform") {
  val ret = model.transform(transSet).cache()
  ret.foreachPartition((_: Iterator[_]) => ())
  ret
}
prediction.select("vendor_id", "passenger_count", "trip_distance", labelName, "prediction").show(10)
val evaluator = new RegressionEvaluator().setLabelCol(labelName)
val (rmse, _) = Benchmark.time("evaluation") {
  evaluator.evaluate(prediction)
}
println(s"RMSE == $rmse")

Elapsed time [transform]: 2.134s
+-------------+---------------+------------------+-----------+------------------+
|    vendor_id|passenger_count|     trip_distance|fare_amount|        prediction|
+-------------+---------------+------------------+-----------+------------------+
|1.559730423E9|            2.0|0.7000000000000001|        5.0|  5.04693603515625|
|1.559730423E9|            3.0|10.700000000000001|       34.0|31.727073669433594|
|1.559730423E9|            1.0|               2.3|       10.0| 9.294451713562012|
|1.559730423E9|            1.0|               4.4|       16.5| 15.05233097076416|
|1.559730423E9|            1.0|               1.5|        7.0| 8.995831489562988|
|1.559730423E9|            1.0|               0.8|        7.5| 6.239481449127197|
|1.559730423E9|            1.0|               1.2|        5.5| 7.339130401611328|
|1.559730423E9|            1.0|               3.0|        2.5|13.403449058532715|
| 4.52563162E8|            1.0|2.3399999999999994|        9.5| 9.

prediction = [vendor_id: double, passenger_count: double ... 15 more fields]
evaluator = RegressionEvaluator: uid=regEval_547b9abc7a3b, metricName=rmse, throughOrigin=false
rmse = 1.9141528880798715


1.9141528880798715

## Save the model to disk and load model
Save the model to disk and then load it to memory. After that use the loaded model to do a new prediction.

In [17]:
model.write.overwrite.save(dataRoot + "/model/taxi")

val modelFromDisk = XGBoostRegressionModel.load(dataRoot + "/model/taxi")
val (results2, _) = Benchmark.time("transform2") {
  modelFromDisk.transform(transSet)
}
results2.select("vendor_id", "passenger_count", "trip_distance", labelName, "prediction").show(5)

Elapsed time [transform2]: 0.025s


modelFromDisk = xgbr_d36c6f5fd67c
results2 = [vendor_id: double, passenger_count: double ... 15 more fields]


+-------------+---------------+------------------+-----------+------------------+
|    vendor_id|passenger_count|     trip_distance|fare_amount|        prediction|
+-------------+---------------+------------------+-----------+------------------+
|1.559730423E9|            2.0|0.7000000000000001|        5.0|  5.04693603515625|
|1.559730423E9|            3.0|10.700000000000001|       34.0|31.727073669433594|
|1.559730423E9|            1.0|               2.3|       10.0| 9.294451713562012|
|1.559730423E9|            1.0|               4.4|       16.5| 15.05233097076416|
|1.559730423E9|            1.0|               1.5|        7.0| 8.995831489562988|
+-------------+---------------+------------------+-----------+------------------+
only showing top 5 rows



[vendor_id: double, passenger_count: double ... 15 more fields]

In [18]:
sparkSession.close()